# Multi-Asset Simulation - Floods
The following notebook contains an example of running the PRISK algorithm on a collection of assets. The inclusion of multiple assets introduces interactions because of the insurance link between assets who share the same insurance.

In [1]:
import numpy as np
import pandas as pd

from prisk.flood import FloodExposure, FloodEntitySim
from prisk.kernel import Kernel
from prisk.asset import PowerPlant
from prisk.insurance import Insurance
from prisk.utils import convert_to_continous_damage

In [2]:
TIME_HORIZON = 10
n_assets = 100
n_insurers = 1

## Data description
The same type of data is required as for the previous notebooks.

In [3]:
power = pd.read_excel("../Data/power.xlsx")
power.drop_duplicates(subset=["Plant / Project name", "Type", "Capacity (MW)", "Country", "Owner"], inplace=True)
power.drop(columns=[2], inplace=True)
power = power#.sample(n_assets, axis=0)
n_assets = power.shape[0]
damage_curves = pd.read_excel("../Data/damage_curves.xlsx")
continuous_curves = convert_to_continous_damage(damage_curves)
return_period_columns = [c for c in power.columns if isinstance(c, int)]

## Simulation
The simulation will consist of 20 assets and 2 insurers. Each firm is assumed to have the same damage curves.

In [4]:
assets = []
insurers = []

for i in range(n_insurers):
    insurer = Insurance(
        name=f"Insurer {i}",
        capital=1e9,
    )
    insurers.append(insurer)

for a in range(n_assets):
    firm = power.iloc[a]
    flood_exposure = [FloodExposure(return_period, firm[return_period]) 
                    for return_period in return_period_columns if firm[return_period] > 0
    ]
    production_path = np.repeat(firm["Capacity (MW)"]*24*365, TIME_HORIZON) # Assume constant production path
    asset = PowerPlant(
        name=firm["Plant / Project name"],
        flood_damage_curve=continuous_curves,
        flood_exposure=flood_exposure,
        production_path=production_path,
        replacement_cost=firm["Value"],
    )
    # Give them a random insurer
    asset.add_insurer(np.random.choice(insurers))
    assets.append(asset)

In [5]:
kernel = Kernel(assets=assets, insurers=insurers)
for asset in assets:
    FloodEntitySim(asset).simulate(time_horizon=TIME_HORIZON, kernel=kernel)
kernel.run(time_horizon=TIME_HORIZON)

## Monte Carlo Sim
Here the monte carlo simulation is done on all of the assets at the same time.

In [6]:
simulations = 250
prisks = pd.DataFrame(columns=[f"Asset {i}" for i in range(n_assets)], 
                      index=[i for i in range(simulations)])
components = pd.DataFrame(
    columns=["asset", "run", "base_value", "replacement",
             "business_disruption", "fair_insurance",
             "insurance_adjustment", "npv", "prisk"]
)


from tqdm import tqdm
for i in tqdm(range(simulations)):
    assets = []
    insurers = []

    for k in range(n_insurers):
        insurer = Insurance(
            name=f"Insurer {k}",
            capital=1e9,
        )
        insurers.append(insurer)

    for a in range(n_assets):
        firm = power.iloc[a]
        flood_exposure = [FloodExposure(return_period, firm[return_period]) 
                        for return_period in return_period_columns if firm[return_period] > 0
        ]
        production_path = np.repeat(firm["Capacity (MW)"]*24*365, TIME_HORIZON) # Assume constant production path
        asset = PowerPlant(
            name=firm["Plant / Project name"],
            flood_damage_curve=continuous_curves,
            flood_exposure=flood_exposure,
            production_path=production_path.copy(),
            replacement_cost=firm["Value"],
        )
        # Give them a random insurer
        asset.add_insurer(np.random.choice(insurers))
        assets.append(asset)
    kernel = Kernel(assets=assets, insurers=insurers)
    for asset in assets:
        FloodEntitySim(asset).simulate(time_horizon=TIME_HORIZON, kernel=kernel)
    kernel.run(time_horizon=TIME_HORIZON, verbose=0)
    for j, asset in enumerate(assets):
        prisk = max(asset.npv/asset.base_value, 0)
        prisks.loc[i, f'Asset {j}'] = prisk
    component = pd.DataFrame({
            "asset": asset.name,
            "run": i,
            "base_value": asset.base_value,
            "replacement": asset.total_replacement_costs,
            "business_disruption": asset.total_business_disruption,
            "fair_insurance": asset.total_fair_insurance_premiums,
            "insurance_adjustment": asset.total_insurance_adjustments,
            "npv": asset.npv,
            "prisk": max(asset.npv/asset.base_value, 0)
        } for asset in assets)
    components = pd.concat([components, component], ignore_index=True)
prisks

  0%|          | 0/250 [00:00<?, ?it/s]

In [ ]:
power.loc[:, 'prisk'] = list(prisks.mean(axis=0))
power.drop(columns=["Latitude", "Longitude"], inplace=True)
power.to_parquet("../results/240617/power_with_prisk_insured.parquet.gzip", compression="gzip")
components.to_parquet("../results/240617/components_insured.parquet.gzip", compression="gzip")
prisks.to_parquet("../results/240617/prisks_insured.parquet.gzip", compression="gzip")
